In [1]:
import pandas as pd

df = pd.read_pickle('../data/final_data.pkl')

In [2]:
# MODEL PREDICTION 'Code_local_type' == 1, MAISON 

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score

# MSE : 3 184 771 589 078.1714 avec 5 colonnes et 100 000 lignes au début du programme (régression linéaire)
# MSE : 9 351 187 946 604.617 avec 11 colonnes et 700 000 lignes au début du programme
# MSE : 9 621 056 123 391.139 en changeant le code type local == 1
# MSE : 2 642 213 419 979.8486 avec 5 colonnes et 100 000 lignes au début du programme (avec year, code postal, et nature mutation) (code local == 1)
# MSE : 2 642 223 129 457.908 avec pareil mais Lasso (few features should be important) (code local == 1)
# MSE : 2 283 126 024 532.157 avec pareil mais Ridge (lot of features should be important) (code local == 1)
# MSE : 1 481 548 894 581.531 avec Ridge mais 300 000 lignes (code local == 1)
# MSE : 1 419 765 756 499.442 avec Lasso mais 300 000 lignes (code local == 1)
# MSE : 15 011 506 135 748.002 avec Lasso mais 1 021 105 lignes (code local == 1)
# MSE : 6 011 506 135 748.002 avec Ridge mais ... lignes (code local == 1)

df_test = df.copy()
# test seulement avec le code local == 1
df_test = df_test[df_test['Code_type_local'] == 1]
df_test = df_test.drop(columns=["Code_type_local"])


print("nombre individus dataframe global : ", len(df_test), "\n")


# test drop colonne
X = df_test.drop(columns=['Valeur_fonciere', 'Commune', 'Section', 'day',
                 'No_plan', 'No_disposition', 'latitude', 'longitude', 'Code_postal', 'month'])
print(X.columns)

X = pd.get_dummies(X, columns=['year', 'Departement'])


y = df_test['Valeur_fonciere']  # Variable cible

# Séparez les données en ensembles d'entraînement et de test (par exemple, 80% pour l'entraînement et 20% pour le test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.5, random_state=42)

# Initialisez le modèle Ridge
modele_ridge = Ridge(alpha=1.0)  # L'alpha est le paramètre de régularisation

modele_ridge.fit(X_train, y_train)
# Entraînez le modèle sur les donné
# Prédisez les valeurs sur l'ensemble de test
predictions = modele_ridge.predict(X_test)

# Calculez l'erreur quadratique moyenne (MSE) pour évaluer la performance du modèle
mse = mean_squared_error(y_test, predictions)

# Affichez l'erreur quadratique moyenne
print("Erreur Quadratique Moyenne (MSE) : ", mse)

# Créez un DataFrame avec les valeurs prédites et les valeurs réelles
resultats = pd.DataFrame(
    {'Valeur_reelle': y_test, 'Valeur_predite': predictions})

# Affichez les valeurs prédites par rapport aux valeurs réelles
print(resultats)
# Calculez le R carré
r2 = r2_score(y_test, predictions)
# Affichez le R carré
print("R carré :", r2)


nombre individus dataframe global :  195285 

Index(['Nombre_de_lots', 'Surface_reelle_bati', 'Nombre_pieces_principales',
       'year', 'Departement', 'Prix_moyen_m2'],
      dtype='object')
Erreur Quadratique Moyenne (MSE) :  8774158725.156519
        Valeur_reelle  Valeur_predite
309420       225900.0   148549.842012
62605        345000.0    98585.124264
276959        16640.0   123869.547406
92664         78000.0    12624.080223
54015        215000.0   162046.443037
...               ...             ...
205146        96116.0   105813.028699
208290       132000.0    80654.955785
164689         2910.0    58721.870987
191641        35000.0    49631.765100
88922        129000.0   151568.108477

[97643 rows x 2 columns]
R carré : 0.38871934879916803


In [3]:
# MODEL PREDICTION 'Code_local_type' == 2, APPARTEMENT

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score


df_test = df.copy()

# test seulement avec le code local == 2
df_test = df_test[df_test['Code_type_local'] == 2]
print(df['Code_type_local'].value_counts())

df_test = df_test.drop(columns=["Code_type_local"])

print("nombre individus dataframe global : ", len(df_test), "\n")


# test drop colonne
X = df_test.drop(columns=['Valeur_fonciere', 'Commune', 'Section', 'day',
                 'No_plan', 'No_disposition', 'latitude', 'longitude', 'Code_postal', 'month'])
print(X.columns)

X = pd.get_dummies(X, columns=['year', 'Departement'])

y = df_test['Valeur_fonciere']  # Variable cible

# Séparez les données en ensembles d'entraînement et de test (par exemple, 80% pour l'entraînement et 20% pour le test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

# Initialisez le modèle Ridge
modele_ridge = Ridge(alpha=1.0)  # L'alpha est le paramètre de régularisation

modele_ridge.fit(X_train, y_train)

# Entraînez le modèle sur les donné
# Prédisez les valeurs sur l'ensemble de test
predictions = modele_ridge.predict(X_test)

# Calculez l'erreur quadratique moyenne (MSE) pour évaluer la performance du modèle
mse = mean_squared_error(y_test, predictions)

# Affichez l'erreur quadratique moyenne
print("Erreur Quadratique Moyenne (MSE) : ", mse)

# Créez un DataFrame avec les valeurs prédites et les valeurs réelles
resultats = pd.DataFrame(
    {'Valeur_reelle': y_test, 'Valeur_predite': predictions})

# Affichez les valeurs prédites par rapport aux valeurs réelles
print(resultats)
# Calculez le R carré
r2 = r2_score(y_test, predictions)
# Affichez le R carré
print("R carré :", r2)


1.0    195285
2.0     93333
3.0     59190
4.0     12528
Name: Code_type_local, dtype: int64
nombre individus dataframe global :  93333 

Index(['Nombre_de_lots', 'Surface_reelle_bati', 'Nombre_pieces_principales',
       'year', 'Departement', 'Prix_moyen_m2'],
      dtype='object')
Erreur Quadratique Moyenne (MSE) :  9851698142.905704
        Valeur_reelle  Valeur_predite
23076        127000.0   125846.440169
237584       105000.0   181720.546810
146256       145000.0   214156.370745
96505         63000.0   117917.799266
204387        76000.0   151126.900993
...               ...             ...
176639       200000.0   175442.101869
31641         41000.0    94312.151112
2775          20000.0   116290.256865
163557       214200.0   226965.024713
229952       250000.0   207202.469193

[18667 rows x 2 columns]
R carré : 0.1850836772416844


In [4]:
# MODEL PREDICTION 'Code_local_type' == 3, DEPENDANCE


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score


df_test = df.copy()

# test seulement avec le code local == 3
df_test = df_test[df_test['Code_type_local'] == 3]
print(df['Code_type_local'].value_counts())

df_test = df_test.drop(columns=["Code_type_local"])

print("nombre individus dataframe global : ", len(df_test), "\n")


# test drop colonne
X = df_test.drop(columns=['Valeur_fonciere', 'Commune', 'Section', 'day',
                 'No_plan', 'No_disposition', 'latitude', 'longitude', 'Code_postal', 'month'])
print(X.columns)

X = pd.get_dummies(X, columns=['year', 'Departement'])

y = df_test['Valeur_fonciere']  # Variable cible

# Séparez les données en ensembles d'entraînement et de test (par exemple, 80% pour l'entraînement et 20% pour le test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

# Initialisez le modèle Ridge
modele_ridge = Ridge(alpha=1.0)  # L'alpha est le paramètre de régularisation

modele_ridge.fit(X_train, y_train)

# Entraînez le modèle sur les donné
# Prédisez les valeurs sur l'ensemble de test
predictions = modele_ridge.predict(X_test)

# Calculez l'erreur quadratique moyenne (MSE) pour évaluer la performance du modèle
mse = mean_squared_error(y_test, predictions)

# Affichez l'erreur quadratique moyenne
print("Erreur Quadratique Moyenne (MSE) : ", mse)

# Créez un DataFrame avec les valeurs prédites et les valeurs réelles
resultats = pd.DataFrame(
    {'Valeur_reelle': y_test, 'Valeur_predite': predictions})

# Affichez les valeurs prédites par rapport aux valeurs réelles
print(resultats)
# Calculez le R carré
r2 = r2_score(y_test, predictions)
# Affichez le R carré
print("R carré :", r2)


1.0    195285
2.0     93333
3.0     59190
4.0     12528
Name: Code_type_local, dtype: int64
nombre individus dataframe global :  59190 

Index(['Nombre_de_lots', 'Surface_reelle_bati', 'Nombre_pieces_principales',
       'year', 'Departement', 'Prix_moyen_m2'],
      dtype='object')


Erreur Quadratique Moyenne (MSE) :  13809529292.802912
        Valeur_reelle  Valeur_predite
206987       327000.0   182026.872338
287931        72500.0   114541.832570
19170        192000.0   171935.917787
251988        18000.0   130776.757463
353412        15000.0    88413.752857
...               ...             ...
243997            5.0   155211.019555
76175         31000.0   173868.956820
103384       249000.0   195270.321805
67305        300000.0   173126.630476
70887         24000.0   140436.000842

[11838 rows x 2 columns]
R carré : 0.10222157114947805
